In [12]:
import requests 
import pandas as pd 
from bs4 import BeautifulSoup 
import datetime
from datetime import datetime, timedelta
import time
import schedule
import h5py

import warnings
warnings.filterwarnings('ignore')

In [13]:
#Create directory
import os

path = './data'
# Check whether the specified path exists or not
isExist = os.path.exists(path)

if not isExist: 
  # Create a new directory because it does not exist 
  os.makedirs(path)
  print("new directory is created!")

In [14]:
today = datetime.now().date()
time_delta_D = timedelta(days=7)
lastweek = today - time_delta_D

today = str(today)
lastweek = str(lastweek)

today

'2022-02-20'

In [15]:
file_url = pd.read_csv('url.csv')
file_url

,url_list
0,https://finance.yahoo.com/cryptocurrencies/
1,https://finance.yahoo.com/gainers
2,https://finance.yahoo.com/currencies
3,https://coinmarketcap.com/nft/collections/


In [16]:
file_url['url_list'].iloc[0]

'https://finance.yahoo.com/cryptocurrencies/'

In [17]:
def web_scrape(url):
    
    Bsoup = BeautifulSoup(requests.get(url).text)
    headers = [header.text for listing in Bsoup.find_all('thead') for header in listing.find_all('th')]
    r_data = {header:[] for header in headers}

    for rows in Bsoup.find_all('tbody'):
      for row in rows.find_all('tr'):
        
        if len(row) != len(headers): continue
        for idx, cell in enumerate(row.find_all('td')):
              r_data[headers[idx]].append(cell.text)
    return pd.DataFrame(r_data)


def job():

    df1 = web_scrape(file_url['url_list'].iloc[0])
    df = df1.iloc[:, 0:3]
    
    h5File = (today + '_d_web_scrape.h5')
    df.to_hdf(h5File, 'w')
    print("wrote hdf5 file done")   
    
#schedule.every(1).minutes.do(job)
#schedule.every(1).hour.do(job)
schedule.every(1).day.at("10:30").do(job)

Every 1 day at 10:30:00 do job() (last run: [never], next run: 2022-02-20 10:30:00)

In [18]:
df1 = web_scrape(file_url['url_list'].iloc[0])
df = df1.iloc[:, 0:3]

df.to_csv(today + '_web_scrape.csv')

h5File = (today + '_web_scrape.h5')
df.to_hdf(h5File, 'w')
print("wrote hdf5 file done")

wrote hdf5 file done


In [19]:
hf = pd.read_hdf(h5File)
hf.head(3)

,Symbol,Name,Price (Intraday)
0,BTC-USD,Bitcoin USD,"40,135.68"
1,ETH-USD,Ethereum USD,"2,759.86"
2,USDT-USD,Tether USD,1.0004


In [20]:
list(hf.keys())

['Symbol', 'Name', 'Price (Intraday)']

In [21]:
hf.shape

(25, 3)

In [22]:
%%writefile web_scraping.py

import requests 
import pandas as pd 
from bs4 import BeautifulSoup 
import datetime
from datetime import datetime, timedelta
import time
import schedule
import h5py

import warnings
warnings.filterwarnings('ignore')

today = datetime.now().date()
today = str(today)

file_url = pd.read_csv('url.csv')

def web_scrape(url):
    
    Bsoup = BeautifulSoup(requests.get(url).text)
    headers = [header.text for listing in Bsoup.find_all('thead') for header in listing.find_all('th')]
    r_data = {header:[] for header in headers}

    for rows in Bsoup.find_all('tbody'):
      for row in rows.find_all('tr'):
        
        if len(row) != len(headers): continue
        for idx, cell in enumerate(row.find_all('td')):
              r_data[headers[idx]].append(cell.text)
    return pd.DataFrame(r_data)


def job():

    df1 = web_scrape(file_url['url_list'].iloc[0])
    df = df1.iloc[:, 0:3]
    
    h5File = (today + '_web_scrape.h5')
    df.to_hdf(h5File, 'w')
    print("wrote hdf5 file done")   
    
#schedule.every().day.at("10:30").do(job)

if __name__ == "__main__":
    job()

Overwriting web_scraping.py


#### Automated task using Task Scheduler on Windows (Run Task Scheduler)
1.Open Start

2.Search for Task Scheduler , click the top result to open the experience.

3.Expand the Task Scheduler Library branch.

4.Select the folder with your tasks.

5.To run a task on demand, right-click it and select the Run option.

6.To edit a task, right-click it and select the Properties options.

7.To delete a task, right-click it and select the Delete option.

In [31]:
from datetime import datetime
with open('timestamps.txt','a+') as file:
    file.write(str(datetime.now()))    